In [1]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import torchvision.models as models
from matplotlib import pyplot as plt

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

In [4]:
train_dataset = datasets.ImageFolder(
    r'C:\Radiologist_helper\datasets\ct_scan\train',
    transform=train_transform
)

val_dataset = datasets.ImageFolder(r'C:\Radiologist_helper\datasets\ct_scan\valid', transform=val_test_transform)
test_dataset = datasets.ImageFolder(r'C:\Radiologist_helper\datasets\ct_scan\test', transform=val_test_transform)

In [5]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [6]:
class CTScanResNet50(nn.Module):
    def __init__(self, num_classes=4, dropout_rate=0.4):
        super().__init__()
        self.model = models.resnet50(weights='DEFAULT')

        # Freeze all layers
        for param in self.model.parameters():
            param.requires_grad = False

        # Optional: unfreeze last block (layer4)
        for param in self.model.layer4.parameters():
            param.requires_grad = True

        # Replace fully connected layer
        in_features = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(in_features, num_classes)
        )

    def forward(self, x):
        return self.model(x)

In [7]:
model = CTScanResNet50(num_classes=4).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4,weight_decay=1e-4)

In [20]:

def train_model(model, criterion, optimizer, epochs=10, patience=6, save_path=r"C:\Radiologist_helper\trained_models\CR_scan_model.pth"):
    start = time.time()
    best_acc = 0.0
    best_loss = float('inf')
    patience_counter = 0

    all_labels, all_predictions = [], []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for batch_num, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

            if (batch_num + 1) % 10 == 0:
                print(f"Batch: {batch_num+1}, Epoch: {epoch+1}, Loss: {loss.item():.4f}")

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{epochs}] - Avg Train Loss: {epoch_loss:.4f}")

        # ---------- Validation ----------
        model.eval()
        correct, total = 0, 0
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                all_labels.extend(labels.cpu().numpy())
                all_predictions.extend(predicted.cpu().numpy())

        val_loss /= len(val_loader.dataset)
        val_acc = 100 * correct / total
        print(f"Validation Loss: {val_loss:.4f} | Accuracy: {val_acc:.2f}%")

        # --- Early stopping and save ---
        if val_loss < best_loss:
            best_loss = val_loss
            best_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), save_path)
            print(f"Best model saved at epoch {epoch+1}")
        else:
            patience_counter += 1
            print(f"No improvement. Patience: {patience_counter}/{patience}")
            if patience_counter >= patience:
                break

    end = time.time()
    print(f"\nTraining complete in {(end - start):.2f}s")
    print(f"Best Validation Accuracy: {best_acc:.2f}%")

    return all_labels, all_predictions

In [21]:
all_labels, all_predictions = train_model(model, criterion, optimizer, epochs=15)

Batch: 10, Epoch: 1, Loss: 0.0288
Batch: 20, Epoch: 1, Loss: 0.9014
Epoch [1/15] - Avg Train Loss: 0.0676
Validation Loss: 5.2539 | Accuracy: 83.33%
Best model saved at epoch 1
Batch: 10, Epoch: 2, Loss: 0.1786
Batch: 20, Epoch: 2, Loss: 0.2898
Epoch [2/15] - Avg Train Loss: 0.0537
Validation Loss: 3.3859 | Accuracy: 81.94%
Best model saved at epoch 2
Batch: 10, Epoch: 3, Loss: 0.0416
Batch: 20, Epoch: 3, Loss: 1.0293
Epoch [3/15] - Avg Train Loss: 0.0466
Validation Loss: 4.7810 | Accuracy: 83.33%
No improvement. Patience: 1/6
Batch: 10, Epoch: 4, Loss: 0.0388
Batch: 20, Epoch: 4, Loss: 0.0649
Epoch [4/15] - Avg Train Loss: 0.0673
Validation Loss: 5.0588 | Accuracy: 88.89%
No improvement. Patience: 2/6
Batch: 10, Epoch: 5, Loss: 0.0192
Batch: 20, Epoch: 5, Loss: 0.1999
Epoch [5/15] - Avg Train Loss: 0.0372
Validation Loss: 4.4806 | Accuracy: 86.11%
No improvement. Patience: 3/6
Batch: 10, Epoch: 6, Loss: 0.0061
Batch: 20, Epoch: 6, Loss: 0.2277
Epoch [6/15] - Avg Train Loss: 0.0394
Val

In [10]:
def evaluate_test(model, test_loader, model_path=r"C:\Radiologist_helper\trained_models\CR_scan_model.pth"):
    device = next(model.parameters()).device
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (preds == labels).sum().item()

    test_acc = 100 * correct / total
    print(f"*** Test Accuracy: {test_acc:.2f}% ***")
    return test_acc


In [11]:
test_acc = evaluate_test(model, test_loader)
print(f"*** Test Accuracy: {test_acc:.2f}% ***")

*** Test Accuracy: 85.08% ***
*** Test Accuracy: 85.08% ***
